In [29]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
# from .feature_utils import time_domain_features
# from .mahalanobis_distance import mahalanobis_distance

from sklearn.preprocessing import MinMaxScaler

# import torch.nn as nn
# import torch.utils.model_zoo as model_zoo
# import torch
# from math import pi
# import torch.nn.functional as F

In [30]:
combined_data = pd.DataFrame(columns=["小時", "分", "秒", "毫秒", "垂直加速度", "水平加速度"])

data_list = []
folder_name = './viberation_dataset/Learning_set/Bearing1_1/'
files = [file for file in sorted(os.listdir(folder_name)) if 'acc' in file]
print(len(files))


2803


In [31]:
files = [file for file in sorted(os.listdir(folder_name)) if 'acc' in file]
    
hrz_data_list = []
for file in files:
    df_hrz = pd.read_csv(f'{folder_name}/{file}', header=None)
    df_hrz = df_hrz[4]
    hrz_data_list.append(df_hrz)
    
hrz_data = np.array(hrz_data_list)
scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_data = scaler.fit_transform(hrz_data)

print(normalized_data.shape)
print(normalized_data)

(2803, 2560)
[[ 0.02160102 -0.11314751 -0.0797902  ... -0.33393533 -0.01268369
  -0.17226443]
 [-0.06632783 -0.19753086 -0.09541346 ... -0.28414173  0.01460812
  -0.11331351]
 [-0.02223634 -0.18951254 -0.11204107 ... -0.30460956 -0.00813506
  -0.09973204]
 ...
 [-0.05463787  0.18111238  0.46657739 ... -0.40608875 -0.09420924
  -0.14979995]
 [-1.         -0.25416826  0.03024216 ... -0.49578603 -0.09963261
  -0.37018684]
 [ 0.03481576 -0.33180603 -0.33980583 ... -0.73142415 -0.30554584
  -0.36130382]]


In [33]:
print(type(normalized_data))
print(normalized_data[0])

<class 'numpy.ndarray'>
[ 0.02160102 -0.11314751 -0.0797902  ... -0.33393533 -0.01268369
 -0.17226443]


In [ ]:
df_hrz = pd.read_csv(f'{folder_name}/{files[0]}', header=None)
# print(df_hrz[4])
hrz_a = df_hrz[4]
print(hrz_a)

plt.figure(figsize=(10, 6))
plt.plot(hrz_a)
plt.xlabel('number of data')
plt.ylabel('Horiz. accel.')
plt.title('raw signal data')
plt.grid(True)
plt.show()

In [ ]:
folder_name = './viberation_dataset/Learning_set/Bearing1_1/'
files = [file for file in sorted(os.listdir(folder_name)) if 'acc' in file]
files = files[0:3]

hrz_data_list = []
for file in files:
    df_hrz = pd.read_csv(f'{folder_name}/{file}', header=None)
    df_hrz = df_hrz[4]
    hrz_data_list.append(df_hrz) 
hrz_data = np.array(hrz_data_list)

scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_data = scaler.fit_transform(hrz_data)

print(normalized_data.shape)
print(normalized_data)

In [ ]:
folder_name = './viberation_dataset/Learning_set/Bearing1_1/'
files = [file for file in sorted(os.listdir(folder_name)) if 'acc' in file]
files = files[0:3]
df_points = 0
for file in files:
    df = pd.read_csv(f'{folder_name}/{file}', header=None)
    df_points += len(df)

label_data = np.linspace(1, 0, df_points)

plot =  True
if plot:
    plt.figure(figsize=(10, 6))
    plt.plot(label_data)
    plt.xlabel('number of data')
    plt.ylabel('Horiz. accel.')
    plt.title('raw signal data')
    plt.grid(True)
    plt.show()

In [ ]:
import torch
import torch.nn as nn

class CNNLSTMModel(nn.Module):
    def __init__(self, num_classes, input_size=2560):
        super(CNNLSTMModel, self).__init()
        
        # 定义CNN部分
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        # 定义LSTM部分
        self.lstm = nn.LSTM(input_size=128, hidden_size=256, num_layers=2, batch_first=True)
        
        # 全连接层
        self.fc = nn.Linear(256, num_classes)
        
    def forward(self, x):
        # 输入x的形状应该是 [batch_size, input_size, num_channels]
        batch_size, input_size, num_channels = x.size()
        
        # CNN部分的前向传播
        x = x.permute(0, 2, 1)  # 将数据变形为 [batch_size, num_channels, input_size]
        x = self.cnn(x)
        x = x.permute(0, 2, 1)  # 恢复形状
        
        # LSTM部分的前向传播
        lstm_out, _ = self.lstm(x)
        
        # 获取LSTM输出的最后一帧
        lstm_out = lstm_out[:, -1, :]
        
        # 全连接层
        output = self.fc(lstm_out)
        
        return output


In [ ]:
import torch

# 创建随机数据
random_data = torch.randn(1, 1, 2560)  # 1个样本，1个通道，2560个时间步
